# Mood Inference Circuit

## Setup

In [1]:
!ls

 Attribution_Patching_Demo.ipynb
 CircuitsVis
 Dockerfile
 LICENSE
 README.md
 __pycache__
 adjective_token_lengths.txt
 ccs.py
 ccs_act_patching.py
 ccs_circuit_analysis.py
 ccs_circuit_attribution.py
 ccs_circuit_path_patching.py
 circuit.md
 circuit_analysis_classification_prompt_experimentation_pythia2_8b.ipynb
 circuit_analysis_contrastive_sentiment_gpt2_small.py
 circuit_analysis_restaurant_review_classification_pythia1_4b.ipynb
'circuit_analysis_sentiment continuation_pythia1_4b.py'
 circuit_analysis_sentiment_classification_pythia1_4b.ipynb
 circuit_analysis_sentiment_classification_pythia1_4b.py
 circuit_analysis_sentiment_continuation_pythia1_4b.ipynb
 circuit_analysis_sentiment_contradiction_pythia1_4b.ipynb
'circuit_analysis_simple single sentiment_gpt2_small.py'
 circuit_analysis_simple_sentiment_gpt2_small.ipynb
 circuit_analysis_simple_sentiment_gpt2_small.py
'circuit_analysis_task comparison_pythia1_4b.ipynb'
 circuit_for_mood_binding_pythia2_8b.ipynb
 circuit_for_mood_

In [2]:
%cd eliciting-latent-sentiment

[Errno 2] No such file or directory: 'eliciting-latent-sentiment'
/notebooks/eliciting-latent-sentiment


In [3]:
!pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
!pip install transformer_lens
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git
#!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
!pip install kaleido
#%pip install git+https://github.com/neelnanda-io/PySvelte.git
#%pip install typeguard==2.13.3

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-xv5h08f4
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-xv5h08f4
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit df9bfc252807e8b1c3a26c3c4796c18342c7fc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 38.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 13.6 MB/s eta 0:00:000

In [3]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [4]:
import os
import pathlib
from typing import List, Optional, Union

import torch
import numpy as np
import yaml
import pickle
import einops
from fancy_einsum import einsum


import circuitsvis as cv

import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import transformer_lens.patching as patching

from torch import Tensor
from tqdm.notebook import tqdm
from jaxtyping import Float, Int, Bool
from typing import List, Optional, Callable, Tuple, Dict, Literal, Set
from rich import print as rprint

from typing import List, Union
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

from functools import partial

from torchtyping import TensorType as TT

from path_patching import Node, IterNode, path_patch, act_patch
from neel_plotly import imshow as imshow_n

from utils.visualization import get_attn_head_patterns, imshow_p, plot_attention_heads, scatter_attention_and_contribution_simple
from utils.visualization import get_attn_pattern, plot_attention

from utils.prompts import get_dataset
from utils.circuit_analysis import get_logit_diff, logit_diff_denoising, logit_diff_noising

/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import torch
torch.set_grad_enabled(False)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
#import plotly
#plotly.offline.init_notebook_mode()

In [6]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Circuit Analysis


### Load Model

In [7]:
from transformers import AutoModelForCausalLM

def load_model(model_name):
    if model_name == "EleutherAI/pythia-6.9b" or model_name == "StabilityAI/stablelm-tuned-alpha-7b":
        source_model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="model_cache").to('cpu').bfloat16()
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False,
            tokenizer=tokenizer,
            hf_model=source_model,
        )
    else:
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False
        )
    return model


In [49]:
model = load_model("EleutherAI/pythia-1.4b")
model.set_use_hook_mlp_in(True)

Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


### Dataset Construction

In [50]:
from utils.prompts import CircularList

In [51]:
names = " John, Anne, Mark, Mary, Peter, Paul, James, Sarah, Mike, Tom, Carl, Sam, Sarah, Carl, Jack"
names = [n[1:] for n in model.to_str_tokens(names)[1::2]]
names = CircularList(names)
# for n in names:
#     print(n)
#     print(model.to_str_tokens(n))
#     print(model.to_str_tokens(" " + n))

In [11]:
orig_prompts = [f"{names[i]} loves parties, and joins them whenever possible. {names[i+1]} hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. {names[i]} feels very" for i in range(len(names))]
name_flip_prompts = [f"{names[i]} loves parties, and joins them whenever possible. {names[i+1]} hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. {names[i+1]} feels very" for i in range(len(names))]

ch_flip_prompts_a = [f"{names[i]} hates parties, and avoids them whenever possible. {names[i+1]} loves parties, and joins them whenever possible. One day, they were invited to a grand gala. {names[i]} feels very" for i in range(len(names))]
ch_flip_prompts_b = [f"{names[i]} hates parties, and avoids them whenever possible. {names[i+1]} loves parties, and joins them whenever possible. One day, they were invited to a grand gala. {names[i+1]} feels very" for i in range(len(names))]

sit_flip_prompts = [f"{names[i]} loves parties, and joins them whenever possible. {names[i+1]} hates parties, and avoids them whenever possible. One day, they were invited to a quiet private library. {names[i]} feels very" for i in range(len(names))]

print(model.to_str_tokens(orig_prompts[0]), len(model.to_str_tokens(orig_prompts[0])))
print(model.to_str_tokens(name_flip_prompts[0]), len(model.to_str_tokens(name_flip_prompts[0])))
print(model.to_str_tokens(ch_flip_prompts_a[0]), len(model.to_str_tokens(ch_flip_prompts_a[0])))
print(model.to_str_tokens(sit_flip_prompts[0]), len(model.to_str_tokens(sit_flip_prompts[0])))

['<|endoftext|>', 'John', ' loves', ' parties', ',', ' and', ' joins', ' them', ' whenever', ' possible', '.', ' Anne', ' hates', ' parties', ',', ' and', ' avoids', ' them', ' whenever', ' possible', '.', ' One', ' day', ',', ' they', ' were', ' invited', ' to', ' a', ' grand', ' gal', 'a', '.', ' John', ' feels', ' very'] 36
['<|endoftext|>', 'John', ' loves', ' parties', ',', ' and', ' joins', ' them', ' whenever', ' possible', '.', ' Anne', ' hates', ' parties', ',', ' and', ' avoids', ' them', ' whenever', ' possible', '.', ' One', ' day', ',', ' they', ' were', ' invited', ' to', ' a', ' grand', ' gal', 'a', '.', ' Anne', ' feels', ' very'] 36
['<|endoftext|>', 'John', ' hates', ' parties', ',', ' and', ' avoids', ' them', ' whenever', ' possible', '.', ' Anne', ' loves', ' parties', ',', ' and', ' joins', ' them', ' whenever', ' possible', '.', ' One', ' day', ',', ' they', ' were', ' invited', ' to', ' a', ' grand', ' gal', 'a', '.', ' John', ' feels', ' very'] 36
['<|endoftext

In [12]:
pos_answers = [" excited", " honored"] #, " amazing", " good"]
neg_answers = [" nervous", " uneasy"] #, " terrible", " bad"]
batch_size = 4 * len(orig_prompts)
n_pairs = 1

def create_dataset_4way(orig_prompts_1, orig_prompts_2, flip_prompts_1, flip_prompts_2):
    clean_prompts = []
    corrupt_prompts = []
    answer_tokens = torch.empty(
            (batch_size, n_pairs, 2), 
            device=device, 
            dtype=torch.long
        )
    for i in range(len(orig_prompts)):
        clean_prompts.append(orig_prompts_1[i])
        clean_prompts.append(flip_prompts_1[i])
        clean_prompts.append(orig_prompts_2[i])
        clean_prompts.append(flip_prompts_2[i])

        corrupt_prompts.append(flip_prompts_1[i])
        corrupt_prompts.append(orig_prompts_1[i])
        corrupt_prompts.append(flip_prompts_2[i])
        corrupt_prompts.append(orig_prompts_2[i])

        for pair_idx in range(n_pairs):
                pos_token = model.to_single_token(pos_answers[pair_idx])
                neg_token = model.to_single_token(neg_answers[pair_idx])
                tokens_dict = {
                    'positive': pos_token, 
                    'negative': neg_token, 
                }
                answer_tokens[i * 4, pair_idx, 0] = tokens_dict['positive']
                answer_tokens[i * 4, pair_idx, 1] = tokens_dict['negative']
                answer_tokens[i * 4 + 1, pair_idx, 0] = tokens_dict['negative']
                answer_tokens[i * 4 + 1, pair_idx, 1] = tokens_dict['positive']
                answer_tokens[i * 4 + 2, pair_idx, 0] = tokens_dict['negative']
                answer_tokens[i * 4 + 2, pair_idx, 1] = tokens_dict['positive']
                answer_tokens[i * 4 + 3, pair_idx, 0] = tokens_dict['positive']
                answer_tokens[i * 4 + 3, pair_idx, 1] = tokens_dict['negative']

    prompts_tokens: Float[Tensor, "batch pos"] = model.to_tokens(
            clean_prompts, prepend_bos=True
        )
    clean_tokens = prompts_tokens.to(device)
    corrupted_tokens = model.to_tokens(
        corrupt_prompts, prepend_bos=True
    ).to(device)

    return clean_prompts, clean_tokens, corrupted_tokens, answer_tokens

all_prompts, clean_tokens, corrupted_tokens, answer_tokens = create_dataset_4way(orig_prompts, name_flip_prompts, ch_flip_prompts_a, ch_flip_prompts_b)

In [13]:
for i in range(clean_tokens.shape[0]):
    print(model.to_string(clean_tokens[i]))
    print(model.to_string(corrupted_tokens[i]))
    print(model.to_str_tokens(answer_tokens[i][0]))
    print("\n")

<|endoftext|>John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. John feels very
<|endoftext|>John hates parties, and avoids them whenever possible. Anne loves parties, and joins them whenever possible. One day, they were invited to a grand gala. John feels very
[' excited', ' nervous']


<|endoftext|>John hates parties, and avoids them whenever possible. Anne loves parties, and joins them whenever possible. One day, they were invited to a grand gala. John feels very
<|endoftext|>John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. John feels very
[' nervous', ' excited']


<|endoftext|>John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. Anne feels very
<|endoftext|>John hates parties, and

In [14]:
len(all_prompts), answer_tokens.shape, clean_tokens.shape, corrupted_tokens.shape

(60, torch.Size([60, 1, 2]), torch.Size([60, 36]), torch.Size([60, 36]))

In [15]:
for i in range(0, 16, 1):
    logits, _ = model.run_with_cache(all_prompts[i])
    log_diff = get_logit_diff(logits, answer_tokens[i].unsqueeze(0))
    #if log_diff < 0.1:
    print(all_prompts[i])
    print(model.to_str_tokens(answer_tokens[i][0]))
    print(log_diff, "\n")

John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. John feels very
[' excited', ' nervous']
tensor(1.0039, device='cuda:0') 

John hates parties, and avoids them whenever possible. Anne loves parties, and joins them whenever possible. One day, they were invited to a grand gala. John feels very
[' nervous', ' excited']
tensor(1.1777, device='cuda:0') 

John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. Anne feels very
[' nervous', ' excited']
tensor(1.0159, device='cuda:0') 

John hates parties, and avoids them whenever possible. Anne loves parties, and joins them whenever possible. One day, they were invited to a grand gala. Anne feels very
[' excited', ' nervous']
tensor(0.3234, device='cuda:0') 

Anne loves parties, and joins them whenever possible. Mark hates parties, and avoids the

In [16]:
all_prompts = all_prompts[:16]
answer_tokens = answer_tokens[:16]
clean_tokens = clean_tokens[:16]
corrupted_tokens = corrupted_tokens[:16]

In [17]:
model.to_string(clean_tokens[3]), model.to_string(corrupted_tokens[3])

('<|endoftext|>John hates parties, and avoids them whenever possible. Anne loves parties, and joins them whenever possible. One day, they were invited to a grand gala. Anne feels very',
 '<|endoftext|>John loves parties, and joins them whenever possible. Anne hates parties, and avoids them whenever possible. One day, they were invited to a grand gala. Anne feels very')

#### Logit Differences

In [18]:
pos_indices = [0, 3, 4, 7, 8, 11, 12, 15]
neg_indices = [1, 2, 5, 6, 9, 10, 13, 14]
name_1_indices = [0, 1, 4, 5, 8, 9, 12, 13]
name_2_indices = [2, 3, 6, 7, 10, 11, 14, 15]
pos_first_indices = [0, 2, 4, 6, 8, 10, 12, 14]
neg_first_indices = [1, 3, 5, 7, 9, 11, 13, 15]

In [19]:
pos_logits, pos_cache = model.run_with_cache(clean_tokens[pos_indices,:])
pos_logit_diff = get_logit_diff(pos_logits, answer_tokens[pos_indices,:])
pos_logit_diff

tensor(0.7755, device='cuda:0')

In [20]:
neg_logits, neg_cache = model.run_with_cache(clean_tokens[neg_indices,:])
neg_logit_diff = get_logit_diff(neg_logits, answer_tokens[neg_indices,:])
neg_logit_diff

tensor(1.0585, device='cuda:0')

In [21]:
name_1_logits, name_1_cache = model.run_with_cache(clean_tokens[name_1_indices, :])
name_1_logit_diff = get_logit_diff(name_1_logits, answer_tokens[name_1_indices, :])
name_1_logit_diff

tensor(1.0751, device='cuda:0')

In [22]:
name_2_logits, name_2_cache = model.run_with_cache(clean_tokens[name_2_indices, :])
name_2_logit_diff = get_logit_diff(name_2_logits, answer_tokens[name_2_indices, :])
name_2_logit_diff

tensor(0.7589, device='cuda:0')

In [23]:
pos_first_logits, pos_first_cache = model.run_with_cache(clean_tokens[pos_first_indices,:])
pos_first_logit_diff = get_logit_diff(pos_first_logits, answer_tokens[pos_first_indices,:])
pos_first_logit_diff

tensor(0.9619, device='cuda:0')

In [24]:
neg_first_logits, neg_first_cache = model.run_with_cache(clean_tokens[neg_first_indices,:])
neg_first_logit_diff = get_logit_diff(neg_first_logits, answer_tokens[neg_first_indices,:])
neg_first_logit_diff

tensor(0.8721, device='cuda:0')

In [25]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = get_logit_diff(clean_logits, answer_tokens, per_prompt=False)
clean_logit_diff

tensor(0.9170, device='cuda:0')

In [26]:
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_tokens, per_prompt=False)
corrupted_logit_diff

tensor(-0.9170, device='cuda:0')

In [27]:
def logit_diff_denoising(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
    flipped_logit_diff: float = corrupted_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    return_tensor: bool = False,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is
    same as on flipped input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = get_logit_diff(logits, answer_tokens)
    ld = ((patched_logit_diff - flipped_logit_diff) / (clean_logit_diff  - flipped_logit_diff))
    if return_tensor:
        return ld
    else:
        return ld.item()


def logit_diff_noising(
        logits: Float[Tensor, "batch seq d_vocab"],
        clean_logit_diff: float = clean_logit_diff,
        corrupted_logit_diff: float = corrupted_logit_diff,
        answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
        return_tensor: bool = False,
    ) -> float:
        '''
        We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
        and -1 when performance has been destroyed (i.e. is same as ABC dataset).
        '''
        patched_logit_diff = get_logit_diff(logits, answer_tokens)
        ld = ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

        if return_tensor:
            return ld
        else:
            return ld.item()

logit_diff_denoising_tensor = partial(logit_diff_denoising, return_tensor=True)
logit_diff_noising_tensor = partial(logit_diff_noising, return_tensor=True)

### Ablation & Swap Experiments

#### Patching Comma Activations

In [28]:
full_stop_patch_pos = [torch.tensor([4,14], dtype=torch.long) for _ in range(len(all_prompts))]
full_stop_patch_pos = torch.stack(full_stop_patch_pos, dim=0).to(device)

In [31]:
results = act_patch(
    model=model,
    orig_input=corrupted_tokens,
    new_cache=clean_cache,
    patching_nodes=IterNode(["v","k"], seq_pos=full_stop_patch_pos), # iterating over all heads' output in all layers
    patching_metric=logit_diff_denoising,
    verbose=True,
)

  0%|          | 0/2048 [00:00<?, ?it/s]

results['v'].shape = (layer=32, head=32)
results['k'].shape = (layer=32, head=32)


In [33]:
imshow_p(
    results['k'] * 100,
    title="Patching output of attention heads (corrupted -> clean)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

In [42]:
results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode('z', seq_pos=full_stop_patch_pos),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in MID_IAM_CM_ATTN_HEADS],
    patching_metric=logit_diff_noising,
    verbose=True
)

  0%|          | 0/1024 [00:00<?, ?it/s]

results['z'].shape = (layer=32, head=32)


In [50]:
imshow_p(
        results["z"][:22] * 100,
        title=f"Direct effect on Intermediate AE Heads' values by comma token patching",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

#### Ablating Commas

In [29]:
from utils.ablation import (
    resample_cache_component,
    mean_over_cache_component,
    zero_cache_component,
    freeze_attn_pattern_hook,
    freeze_attn_head_pos_hook,
    freeze_mlp_pos_hook,
    freeze_layer_pos_hook,
    ablate_attn_head_pos_hook,
    ablate_mlp_pos_hook,
    ablate_layer_pos_hook
)

##### Swapping Attention Head Output

In [30]:
heads_to_ablate = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
print(f"Heads to ablate: {heads_to_ablate}")

def freeze_attn_hook(pattern, hook, layer=0, head_idx=0):
    pattern[:, head_idx, :, :] = clean_cache[f"blocks.{layer}.attn.hook_pattern"][:, head_idx, :, :] 
    pattern[:, head_idx, :, :] = clean_cache[f"blocks.{layer}.attn.hook_pattern"][:, head_idx, :, :]
    return pattern

for layer, head in heads_to_freeze:
    freeze_attn = partial(freeze_attn_hook, layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_pattern.add_hook(freeze_attn)

def swap_commas_hook(z, hook, layer=0, head_idx=0):
    #z[:, 4, head_idx, :] = resample_cache_component(corrupted_cache[f"blocks.{layer}.attn.hook_v"][:, 4, head_idx, :])
    #z[:, 14, head_idx, :] = resample_cache_component(corrupted_cache[f"blocks.{layer}.attn.hook_v"][:, 14, head_idx, :])

    z[:, 4, head_idx, :] = corrupted_cache[f"blocks.{layer}.attn.hook_v"][:, 4, head_idx, :]
    z[:, 14, head_idx, :] = corrupted_cache[f"blocks.{layer}.attn.hook_v"][:, 14, head_idx, :]
    #z[:, 10, head_idx, :] = 0 #temp_z[:, 20, head_idx, :]
    #z[:, 20, head_idx, :] = 0 #temp_z[:, 10, head_idx, :]
    return z

for layer, head in heads_to_ablate:
    swap_head_hook = partial(swap_commas_hook, layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_v.add_hook(swap_head_hook)



ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
model.reset_hooks()

print(f"Original logit diff: {get_logit_diff(clean_logits, answer_tokens).item():.4f}")
print(f"Post ablation logit diff: {get_logit_diff(ablated_logits, answer_tokens).item()}")

Heads to ablate: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 1

In [31]:
heads_to_ablate = [(layer, head) for layer in range(23, 32) for head in range(model.cfg.n_heads)]
print(f"Heads to ablate: {heads_to_ablate}")
def ablate_precommas_hook(z, hook, head_idx=0):
    temp_z = z.clone()
    #z[:, 1:4, head_idx, :] = temp_z[:, 11:14, head_idx, :]
    #z[:, 11:14, head_idx, :] = temp_z[:, 1:4, head_idx, :]
    z[:, 1:4, head_idx, :] = corrupted_cache[f"blocks.{layer}.attn.hook_z"][:, 11:14, head_idx, :] 
    z[:, 11:14, head_idx, :] = corrupted_cache[f"blocks.{layer}.attn.hook_z"][:, 1:4, head_idx, :] 
    #z[:, 10, head_idx, :] = 0 #temp_z[:, 20, head_idx, :]
    #z[:, 20, head_idx, :] = 0 #temp_z[:, 10, head_idx, :]
    return z

for layer, head in heads_to_ablate:
    ablate_precommas = partial(ablate_precommas_hook, head_idx=head)
    model.blocks[layer].attn.hook_z.add_hook(ablate_precommas)

ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
model.reset_hooks()

print(f"Original logit diff: {get_logit_diff(clean_logits, answer_tokens).item():.4f}")
print(f"Post ablation logit diff: {get_logit_diff(ablated_logits, answer_tokens).item()}")

Heads to ablate: [(23, 0), (23, 1), (23, 2), (23, 3), (23, 4), (23, 5), (23, 6), (23, 7), (23, 8), (23, 9), (23, 10), (23, 11), (23, 12), (23, 13), (23, 14), (23, 15), (23, 16), (23, 17), (23, 18), (23, 19), (23, 20), (23, 21), (23, 22), (23, 23), (23, 24), (23, 25), (23, 26), (23, 27), (23, 28), (23, 29), (23, 30), (23, 31), (24, 0), (24, 1), (24, 2), (24, 3), (24, 4), (24, 5), (24, 6), (24, 7), (24, 8), (24, 9), (24, 10), (24, 11), (24, 12), (24, 13), (24, 14), (24, 15), (24, 16), (24, 17), (24, 18), (24, 19), (24, 20), (24, 21), (24, 22), (24, 23), (24, 24), (24, 25), (24, 26), (24, 27), (24, 28), (24, 29), (24, 30), (24, 31), (25, 0), (25, 1), (25, 2), (25, 3), (25, 4), (25, 5), (25, 6), (25, 7), (25, 8), (25, 9), (25, 10), (25, 11), (25, 12), (25, 13), (25, 14), (25, 15), (25, 16), (25, 17), (25, 18), (25, 19), (25, 20), (25, 21), (25, 22), (25, 23), (25, 24), (25, 25), (25, 26), (25, 27), (25, 28), (25, 29), (25, 30), (25, 31), (26, 0), (26, 1), (26, 2), (26, 3), (26, 4), (26, 5)

##### Resample-Ablating Non-Commas with Frozen Attention

In [40]:
comma_pos = torch.tensor([4, 10, 14, 20])
precomma_pos = torch.tensor([1, 2, 3, 11, 12, 13])

In [41]:
heads_to_ablate = [(layer, head) for layer in range(9, 15) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
layers_to_freeze = [layer for layer in range(0, 32)]
print(f"Heads to ablate: {heads_to_ablate}")

Heads to ablate: [(9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (9, 19), (9, 20), (9, 21), (9, 22), (9, 23), (9, 24), (9, 25), (9, 26), (9, 27), (9, 28), (9, 29), (9, 30), (9, 31), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (10, 20), (10, 21), (10, 22), (10, 23), (10, 24), (10, 25), (10, 26), (10, 27), (10, 28), (10, 29), (10, 30), (10, 31), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13), (11, 14), (11, 15), (11, 16), (11, 17), (11, 18), (11, 19), (11, 20), (11, 21), (11, 22), (11, 23), (11, 24), (11, 25), (11, 26), (11, 27), (11, 28), (11, 29), (11, 30), (11, 31), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12

In [42]:
def run_precomma_ablation_experiment(heads_to_ablate, heads_to_freeze):
    comma_pos = torch.tensor([4, 10, 14, 20])
    precomma_pos = torch.tensor([1, 2, 3, 11, 12, 13])

    # freeze attention patterns
    for layer, head in heads_to_freeze:
        freeze_attn = partial(freeze_attn_pattern_hook, cache=clean_cache, layer=layer, head_idx=head)
        model.blocks[layer].attn.hook_pattern.add_hook(freeze_attn)

    # freeze comma attn values
    for layer, head in heads_to_freeze:
        freeze_commas = partial(freeze_attn_head_pos_hook, cache=clean_cache, component_type="hook_v", pos=comma_pos, layer=layer, head_idx=head)
        model.blocks[layer].attn.hook_v.add_hook(freeze_commas)

    # freeze comma mlp_out positions
    # for layer in layers_to_freeze:
    #     freeze_comma_mlps = partial(freeze_layer_pos_hook, cache=clean_cache, component_type="hook_mlp_out", pos=comma_pos, layer=layer)
    #     model.blocks[layer].hook_mlp_out.add_hook(freeze_comma_mlps)

    # freeze comma resid_post
    # for layer in layers_to_freeze:
    #     freeze_comma_resid = partial(freeze_layer_pos_hook, cache=clean_cache, component_type="hook_resid_post", pos=comma_pos, layer=layer)
    #     model.blocks[layer].hook_resid_post.add_hook(freeze_comma_resid)

    # ablate pre-comma values
    for layer, head in heads_to_ablate:
        ablate_precommas = partial(ablate_attn_head_pos_hook, cache=corrupted_cache, ablation_func=None, component_type="hook_v", pos=precomma_pos, layer=layer, head_idx=head)
        model.blocks[layer].attn.hook_v.add_hook(ablate_precommas)

    ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
    model.reset_hooks()

    return get_logit_diff(ablated_logits, answer_tokens).item(), get_logit_diff(ablated_logits, answer_tokens, per_prompt=True), get_logit_diff(clean_logits, answer_tokens, per_prompt=True)

In [43]:
ablated_logit_diff, ablated_ld_list, clean_ld_list = run_precomma_ablation_experiment(heads_to_ablate, heads_to_freeze)
model.reset_hooks()

print(f"Original logit diff: {get_logit_diff(clean_logits, answer_tokens).item():.4f}")
print(f"Post ablation logit diff: {ablated_logit_diff:.4f}")
print(f"Logit diff % change: {(ablated_logit_diff - get_logit_diff(clean_logits, answer_tokens).item()) / get_logit_diff(clean_logits, answer_tokens).item():.2%}")

Original logit diff: 0.9170
Post ablation logit diff: 0.5683
Logit diff % change: -38.02%


In [34]:
model.reset_hooks()

##### Resample-Ablating Non-Commas with Frozen Values

In [35]:
heads_to_ablate = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
layers_to_freeze = [layer for layer in range(0, 32)]
print(f"Heads to ablate: {heads_to_ablate}")

Heads to ablate: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 1

In [36]:
comma_pos = torch.tensor([4, 10, 14, 20])
precomma_pos = torch.tensor([[1, 2, 3], [11, 12, 13]])

# freeze values
for layer, head in heads_to_freeze:
    freeze_values = partial(freeze_attn_head_pos_hook, cache=clean_cache, component_type="hook_v", pos="each", layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_v.add_hook(freeze_values)

# freeze comma attn values
for layer, head in heads_to_freeze:
    freeze_commas = partial(freeze_attn_head_pos_hook, cache=clean_cache, component_type="hook_z", pos=comma_pos, layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_z.add_hook(freeze_commas)

# freeze comma mlp_out positions
# for layer in layers_to_freeze:
#     freeze_comma_mlps = partial(freeze_layer_pos_hook, cache=clean_cache, component_type="hook_mlp_out", pos=comma_pos, layer=layer)
#     model.blocks[layer].hook_mlp_out.add_hook(freeze_comma_mlps)

# freeze comma resid_post
# for layer in layers_to_freeze:
#     freeze_comma_resid = partial(freeze_layer_pos_hook, cache=clean_cache, component_type="hook_resid_post", pos=comma_pos, layer=layer)
#     model.blocks[layer].hook_resid_post.add_hook(freeze_comma_mlps)

# ablate pre-comma values
for layer, head in heads_to_ablate:
    ablate_precommas = partial(ablate_attn_head_pos_hook, cache=corrupted_cache, ablation_func=None, component_type="hook_q", pos=precomma_pos, layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_q.add_hook(ablate_precommas) 

# ablate pre-comma values
for layer, head in heads_to_ablate:
    ablate_precommas = partial(ablate_attn_head_pos_hook, cache=corrupted_cache, ablation_func=None, component_type="hook_k", pos=precomma_pos, layer=layer, head_idx=head)
    model.blocks[layer].attn.hook_k.add_hook(ablate_precommas) 


In [39]:
ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
model.reset_hooks()

print(f"Original logit diff: {get_logit_diff(clean_logits, answer_tokens).item():.4f}")
print(f"Post ablation logit diff: {get_logit_diff(ablated_logits, answer_tokens).item()}")
print(f"Logit diff % change: {(ablated_logit_diff - get_logit_diff(clean_logits, answer_tokens).item()) / get_logit_diff(clean_logits, answer_tokens).item():.2%}")

Original logit diff: 0.9170
Post ablation logit diff: 0.9169943928718567
Logit diff % change: -38.02%


In [38]:
model.reset_hooks()

##### Resample-Ablating Commas

In [44]:
heads_to_ablate = [(layer, head) for layer in range(9, 15) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(0, 32) for head in range(model.cfg.n_heads)]
print(f"Heads to ablate: {heads_to_ablate}")

Heads to ablate: [(9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (9, 19), (9, 20), (9, 21), (9, 22), (9, 23), (9, 24), (9, 25), (9, 26), (9, 27), (9, 28), (9, 29), (9, 30), (9, 31), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (10, 20), (10, 21), (10, 22), (10, 23), (10, 24), (10, 25), (10, 26), (10, 27), (10, 28), (10, 29), (10, 30), (10, 31), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13), (11, 14), (11, 15), (11, 16), (11, 17), (11, 18), (11, 19), (11, 20), (11, 21), (11, 22), (11, 23), (11, 24), (11, 25), (11, 26), (11, 27), (11, 28), (11, 29), (11, 30), (11, 31), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12

In [45]:
comma_pos = torch.tensor([4, 10, 14, 20])
precomma_pos = torch.tensor([[1, 2, 3, 11, 12, 13]])

In [46]:
def run_comma_ablation_experiment(heads_to_ablate, heads_to_freeze):
    comma_pos = torch.tensor([4, 10, 14, 20])
    precomma_pos = torch.tensor([[1, 2, 3, 11, 12, 13]])

    # freeze attention patterns
    for layer, head in heads_to_freeze:
        freeze_attn = partial(freeze_attn_pattern_hook, cache=clean_cache, layer=layer, head_idx=head)
        model.blocks[layer].attn.hook_pattern.add_hook(freeze_attn)

    # freeze precomma attn values
    # for layer, head in heads_to_freeze:
    #     freeze_precommas = partial(freeze_attn_head_pos_hook, cache=clean_cache, component_type="hook_v", pos=precomma_pos, layer=layer, head_idx=head)
    #     model.blocks[layer].attn.hook_v.add_hook(freeze_precommas)

    # freeze comma mlp_out positions
    # for layer in layers_to_freeze:
    #     freeze_comma_mlps = partial(freeze_layer_pos_hook, cache=clean_cache, component_type="hook_mlp_out", pos=comma_pos, layer=layer)
    #     model.blocks[layer].hook_mlp_out.add_hook(freeze_comma_mlps)

    # ablate comma attn values
    for layer, head in heads_to_ablate:
        ablate_commas = partial(ablate_attn_head_pos_hook, cache=corrupted_cache, ablation_func=None, component_type="hook_v", pos=comma_pos, layer=layer, head_idx=head)
        model.blocks[layer].attn.hook_v.add_hook(ablate_commas)

    ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
    model.reset_hooks()

    return get_logit_diff(ablated_logits, answer_tokens).item(), get_logit_diff(ablated_logits, answer_tokens, per_prompt=True), get_logit_diff(clean_logits, answer_tokens, per_prompt=True)

In [47]:
ablated_logit_diff, ablated_ld_list, clean_ld_list = run_comma_ablation_experiment(heads_to_ablate, heads_to_freeze)
model.reset_hooks()

print(f"Original logit diff: {get_logit_diff(clean_logits, answer_tokens).item():.4f}")
print(f"Post ablation logit diff: {ablated_logit_diff:.4f}")
print(f"Logit diff % change: {(ablated_logit_diff - get_logit_diff(clean_logits, answer_tokens).item()) / get_logit_diff(clean_logits, answer_tokens).item():.2%}")

Original logit diff: 0.9170
Post ablation logit diff: 0.5873
Logit diff % change: -35.96%


In [48]:
model.reset_hooks()